In [1]:
## Import dependencies

import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import time
import optuna
import pandas as pd
from torch.cuda.amp import GradScaler, autocast

# Set default plot size
plt.rcParams["figure.figsize"] = (30,20)

## Optimize CNN architecture and hyperparameters on dataset of 44 metabolites

In [2]:
# Define number of epochs used later in training (this will have to be redefined later......)
num_epochs = 500

# Name variable used for saving model metrics, name should reflect model used, dataset used, and other information such as # of epochs
ModelName = "CNN_44Met_Optuna_ForManuscript" + str(num_epochs) +"ep"

# Set the random seed
os.chdir('/home/htjhnson/Desktop/DL-NMR-Optimization/ModelPerformanceMetrics/') 
seed = 1 
torch.manual_seed(seed)
np.save(ModelName + "_Seed.npy", seed)

In [3]:
# Load training and testing datasets, validation datasets, and representative example spectra 

# Switch to directory containing datasets
os.chdir('/home/htjhnson/Desktop/DL-NMR-Optimization/GeneratedDataAndVariables')

# Load training data and max value from testing and training datasets
spectra = np.load('Dataset44_ExtendedRange_MoreLeftOut_Combined1Distribution_ForManuscript_Spec.npy')
conc1 = np.load('Dataset44_ExtendedRange_MoreLeftOut_Combined1Distribution_ForManuscript_Conc.npy')

spectraVal = np.load('Dataset44_ExtendedRange_MoreLeftOut_Combined1Distribution_ForManuscript_Val_Spec.npy')
concVal = np.load('Dataset44_ExtendedRange_MoreLeftOut_Combined1Distribution_ForManuscript_Val_Conc.npy')

In [4]:
## Prepare to switch data from CPU to GPU

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    device = torch.device("cuda")          # A CUDA device object
    print("Using GPU for training.")
else:
    device = torch.device("cpu")           # A CPU object
    print("CUDA is not available. Using CPU for training.")

Using GPU for training.


In [5]:
## Set up data for testing and training

# Split into testing and training data
X_train, X_test, y_train, y_test = train_test_split(spectra, conc1, test_size = 0.2, random_state = 1)

# Tensorize and prepare datasets
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()


# Move the input data to the GPU device
X_train = X_train.to(device)
X_test = X_test.to(device)
spectraVal = torch.tensor(spectraVal).float().to(device)   # Confusing names, these spectra are the 5000 spectra generated like the training dataset
#ValSpectra = torch.tensor(ValSpectra).float().to(device)   # Confusing names, these spectra are the 10 representative example spectra

# Move the target data to the GPU device
y_train = y_train.to(device)
y_test = y_test.to(device)
concVal = torch.tensor(concVal).float().to(device)
#ValConc = torch.tensor(ValConc).float().to(device)

# More data prep?
datasets = torch.utils.data.TensorDataset(X_train, y_train)
Test_datasets = torch.utils.data.TensorDataset(X_test, y_test)
Val_datasets = torch.utils.data.TensorDataset(spectraVal, concVal)
#train_iter = torch.utils.data.DataLoader(datasets, batch_size = 128, shuffle=True)
#test_iter = torch.utils.data.DataLoader(Test_datasets, batch_size = 128, shuffle=True)

In [6]:
del X_train
del X_test
del y_train
del y_test
del spectra
del conc1
del spectraVal
del concVal

In [7]:
class QuantileLoss(nn.Module):
    def __init__(self, quantile=0.5):
        super(QuantileLoss, self).__init__()
        self.quantile = quantile

    def forward(self, y_pred, y_true):
        error = y_true - y_pred
        loss = torch.mean(torch.max(self.quantile * error, (self.quantile - 1) * error))
        return loss
    
    
    
# MAPE loss function for directly comparing models despite loss function used
class MAPELoss(nn.Module):
    def __init__(self):
        super(MAPELoss, self).__init__()

    def forward(self, y_pred, y_true):
        loss = torch.mean(torch.abs((y_true - y_pred) / y_true))
        return loss * 100  # To get percentage

In [8]:
## Optimization function

# Switch to directory for saving weights
os.chdir('/home/htjhnson/Desktop/DL-NMR-Optimization/ModelOptimizations/BestWeights')

# Define file name for best model weights
save_path = ModelName + '_Params.pt'


# Define the objective function to be minimized.

def objective(trial):
    accumulation_steps = 4
    n_conv_layers = trial.suggest_int('num_conv_layers', 1, 4)
    kernel_size = trial.suggest_int('kernel_size', 3, 12)
    num_channels = trial.suggest_int('num_channels', 5, 50)
    pooling_type = trial.suggest_categorical('pooling_type', ['none', 'max', 'avg'])
    conv_stride = trial.suggest_int('conv_stride', 1, 10)
    n_fc_layers = 1  # Number of fully connected layers
    feedforward_size = trial.suggest_int("feedforward_size", 100, 250)
    pool_kernel = 2
    if pooling_type != 'none':
        pool_stride = trial.suggest_int('pool_stride', 1, 2)

    layers = []
    in_channels = 1  # Assuming 1 input channel
    in_features = 46000  # Number of input features
    dilation = 1
    padding = 1

    for i in range(n_conv_layers):
        out_channels = num_channels
        layers.append(nn.Conv1d(in_channels, out_channels, kernel_size, stride=conv_stride, dilation=dilation, padding=padding))
        layers.append(nn.ReLU())
        in_features = ((in_features + 2*padding - dilation*(kernel_size - 1) - 1) // conv_stride) + 1
        in_channels = out_channels
        
        if pooling_type != 'none':
            if pooling_type == 'max':
                layers.append(nn.MaxPool1d(pool_kernel, stride=pool_stride, padding=padding))
            else:
                layers.append(nn.AvgPool1d(pool_kernel, stride=pool_stride, padding=padding))
            in_features = ((in_features + 2*padding - pool_kernel) // pool_stride) + 1

    layers.append(nn.Flatten())
    
    for _ in range(n_fc_layers):
        layers.append(nn.Linear(in_features * in_channels, feedforward_size))
        layers.append(nn.ReLU())
        in_features = feedforward_size
        in_channels = 1

    layers.append(nn.Linear(in_features, 44))   # For quantifying 44 metabolites

    model = nn.Sequential(*layers).to(device)
    
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-2)
    reg_strength = trial.suggest_float('reg_strength', 1e-6, 1e-2)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=reg_strength)
    
    bs = trial.suggest_int("bs", 16, 128)
    train_loader = torch.utils.data.DataLoader(datasets, batch_size=bs, shuffle=True)
    test_loader = torch.utils.data.DataLoader(Test_datasets, batch_size=bs, shuffle=False)
    val_loader = torch.utils.data.DataLoader(Val_datasets, batch_size=bs, shuffle=False)

    num_epochs = 500
    criterion = QuantileLoss()
    criterion2 = MAPELoss()
    best_test_loss = float('inf')
    start = time.time()

    for epoch in range(num_epochs):
        start2 = time.time()
        running_train_loss = 0.0
        running_test_loss = 0.0
                
        model.train()
        scaler = GradScaler()
        optimizer.zero_grad()
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.unsqueeze(1)
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss = loss / accumulation_steps
            running_train_loss += loss.item() * inputs.size(0)
            scaler.scale(loss).backward()

            if (batch_idx + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
        
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                inputs = inputs.unsqueeze(1)
                with autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                running_test_loss += loss.item() * inputs.size(0)

        running_test_loss2 = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                inputs = inputs.unsqueeze(1)
                with autocast():
                    outputs = model(inputs)
                    loss2 = criterion2(outputs, labels)
                running_test_loss2 += loss2.item() * inputs.size(0)
        
        if epoch % 1 == 0:
            end_time2 = time.time()
            print(f'Epoch {epoch + 1}/{num_epochs} | Train Loss: {running_train_loss:.3f} | Test Loss: {running_test_loss:.3f} | Test Loss [MAPE]: {running_test_loss2:.3f}', "--time--", end_time2-start2)

        if running_test_loss2 < best_test_loss:
            best_test_loss = running_test_loss2
            
        trial.report(running_test_loss, epoch)
        
        '''
        if epoch > 15 and trial.should_prune():
            raise optuna.TrialPruned()
        '''
        
        ## Double threshold pruner
        if epoch > 15 and running_test_loss > 5000 and trial.should_prune():
            raise optuna.TrialPruned()
        elif epoch > 100 and running_test_loss > 1000 and trial.should_prune():
            raise optuna.TrialPruned()

            
    end_time = time.time()
    print(end_time - start)   

    return best_test_loss

study_name = "CNN_Optuna_ForManuscript"
storage = "sqlite:///CNN_Optuna_ForManuscript.db"  # SQLite database as storage
study = optuna.create_study(direction='minimize',  sampler=optuna.samplers.TPESampler(seed=3), 
                            study_name=study_name, storage=storage,
                            pruner=optuna.pruners.ThresholdPruner(upper=500.0))

study.optimize(objective, n_trials=150)

[I 2024-08-01 21:58:40,028] A new study created in RDB with name: CNN_Optuna_ForManuscript


Epoch 1/500 | Train Loss: 123779.030 | Test Loss: 5162.517 | Test Loss [MAPE]: 429163.339 --time-- 5.798574447631836
Epoch 2/500 | Train Loss: 5106.874 | Test Loss: 5162.334 | Test Loss [MAPE]: 450108.679 --time-- 4.2054924964904785
Epoch 3/500 | Train Loss: 5106.755 | Test Loss: 5162.389 | Test Loss [MAPE]: 452748.733 --time-- 4.2149834632873535
Epoch 4/500 | Train Loss: 5106.983 | Test Loss: 5161.981 | Test Loss [MAPE]: 426240.936 --time-- 4.226120471954346
Epoch 5/500 | Train Loss: 5106.918 | Test Loss: 5162.432 | Test Loss [MAPE]: 435016.414 --time-- 4.226314067840576
Epoch 6/500 | Train Loss: 5108.415 | Test Loss: 5163.527 | Test Loss [MAPE]: 435418.237 --time-- 4.192160606384277
Epoch 7/500 | Train Loss: 5107.464 | Test Loss: 5162.797 | Test Loss [MAPE]: 458230.844 --time-- 4.23551869392395
Epoch 8/500 | Train Loss: 5107.382 | Test Loss: 5162.763 | Test Loss [MAPE]: 454800.070 --time-- 4.237369060516357
Epoch 9/500 | Train Loss: 5107.264 | Test Loss: 5163.616 | Test Loss [MAPE]: 

[I 2024-08-01 21:59:55,042] Trial 0 pruned. 


Epoch 17/500 | Train Loss: 5107.315 | Test Loss: 5163.472 | Test Loss [MAPE]: 457806.704 --time-- 4.184074640274048
Epoch 1/500 | Train Loss: 174200.981 | Test Loss: 5164.122 | Test Loss [MAPE]: 5816882.144 --time-- 2.5879898071289062
Epoch 2/500 | Train Loss: 4613.134 | Test Loss: 4229.693 | Test Loss [MAPE]: 3905282.417 --time-- 2.5410165786743164
Epoch 3/500 | Train Loss: 4169.876 | Test Loss: 4119.167 | Test Loss [MAPE]: 2893807.302 --time-- 2.5351552963256836
Epoch 4/500 | Train Loss: 4101.899 | Test Loss: 4076.439 | Test Loss [MAPE]: 2335883.608 --time-- 2.5375611782073975
Epoch 5/500 | Train Loss: 4037.838 | Test Loss: 4017.628 | Test Loss [MAPE]: 3135208.531 --time-- 2.5352861881256104
Epoch 6/500 | Train Loss: 3994.860 | Test Loss: 3983.011 | Test Loss [MAPE]: 3043754.544 --time-- 2.5381650924682617
Epoch 7/500 | Train Loss: 3954.501 | Test Loss: 3974.350 | Test Loss [MAPE]: 2484265.388 --time-- 2.5377488136291504
Epoch 8/500 | Train Loss: 3910.934 | Test Loss: 3895.837 | Test

[I 2024-08-01 22:01:17,188] Trial 1 pruned. 


...

Epoch 17/500 | Train Loss: 5106.465 | Test Loss: 5161.772 | Test Loss [MAPE]: 425603.924 --time-- 3.8731448650360107
Epoch 1/500 | Train Loss: inf | Test Loss: 5193.844 | Test Loss [MAPE]: 796712.190 --time-- 4.071003198623657
Epoch 2/500 | Train Loss: 5115.113 | Test Loss: 5162.702 | Test Loss [MAPE]: 444118.408 --time-- 4.057260513305664
Epoch 3/500 | Train Loss: 5106.682 | Test Loss: 5161.795 | Test Loss [MAPE]: 427256.776 --time-- 4.053861618041992
Epoch 4/500 | Train Loss: 5106.490 | Test Loss: 5161.836 | Test Loss [MAPE]: 428864.367 --time-- 4.053895711898804
Epoch 5/500 | Train Loss: 5106.591 | Test Loss: 5162.011 | Test Loss [MAPE]: 420778.354 --time-- 4.0531346797943115
Epoch 6/500 | Train Loss: 5106.591 | Test Loss: 5162.037 | Test Loss [MAPE]: 417059.767 --time-- 4.055791616439819
Epoch 7/500 | Train Loss: 5106.623 | Test Loss: 5161.947 | Test Loss [MAPE]: 420451.361 --time-- 4.054313659667969
Epoch 8/500 | Train Loss: 5106.652 | Test Loss: 5161.994 | Test Loss [MAPE]: 42231

[I 2024-08-02 10:00:22,522] Trial 60 pruned. 


Epoch 17/500 | Train Loss: 5106.769 | Test Loss: 5162.083 | Test Loss [MAPE]: 434465.666 --time-- 4.0550923347473145
Epoch 1/500 | Train Loss: 5251.023 | Test Loss: 5163.047 | Test Loss [MAPE]: 514526.562 --time-- 12.74380373954773
Epoch 2/500 | Train Loss: 4805.460 | Test Loss: 4328.338 | Test Loss [MAPE]: 1385848.699 --time-- 12.723872900009155
Epoch 3/500 | Train Loss: 4177.834 | Test Loss: 4081.738 | Test Loss [MAPE]: 2391050.030 --time-- 12.716630220413208
Epoch 4/500 | Train Loss: 3992.950 | Test Loss: 3905.013 | Test Loss [MAPE]: 3311636.358 --time-- 12.715004920959473
Epoch 5/500 | Train Loss: 3820.317 | Test Loss: 3747.006 | Test Loss [MAPE]: 5236569.251 --time-- 12.72046685218811
Epoch 6/500 | Train Loss: 3700.429 | Test Loss: 3635.573 | Test Loss [MAPE]: 5798337.833 --time-- 12.720614433288574
Epoch 7/500 | Train Loss: 3594.693 | Test Loss: 3551.831 | Test Loss [MAPE]: 6042438.919 --time-- 12.72122597694397
Epoch 8/500 | Train Loss: 3528.476 | Test Loss: 3469.649 | Test Loss

[W 2024-08-02 10:04:02,724] Trial 61 failed with parameters: {'num_conv_layers': 4, 'kernel_size': 7, 'num_channels': 50, 'pooling_type': 'max', 'conv_stride': 1, 'feedforward_size': 111, 'pool_stride': 2, 'learning_rate': 0.0027529748933444393, 'reg_strength': 0.0020272022132670014, 'bs': 124} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/htjhnson/anaconda3/lib/python3.11/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_66682/1893709827.py", line 87, in objective
    running_train_loss += loss.item() * inputs.size(0)
                          ^^^^^^^^^^^
KeyboardInterrupt
[W 2024-08-02 10:04:02,726] Trial 61 failed with value None.


KeyboardInterrupt: 

In [ ]:
# Switch to directory for saving weights
os.chdir('/home/htjhnson/Desktop/DL-NMR-Optimization/ModelOptimizations/BestWeights/')

loaded_study = optuna.load_study(study_name="CNN_Optuna_ForManuscript", storage="sqlite:///CNN_Optuna_ForManuscript.db")

In [ ]:
loaded_study.trials_dataframe()

In [ ]:
loaded_study.best_value

In [ ]:
loaded_study.best_params

In [ ]:
for trial in loaded_study.trials:
    print(trial)

In [ ]:
trials_df = loaded_study.trials_dataframe()
print(trials_df)

In [ ]:
for trial in loaded_study.trials:
    print(f"Trial number: {trial.number}")
    print(f"  Params: {trial.params}")
    print(f"  Value: {trial.value}")
    print(f"  State: {trial.state}")

In [ ]:
# Get the best parameters and best value from the study
best_params = loaded_study.best_params
best_value = loaded_study.best_value

# Create a dictionary to store hyperparameters and best loss
data = {
    'Hyperparameter': list(best_params.keys()) + ['Best Loss'],
    'Value': list(best_params.values()) + [best_value]
}

# Create a pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_contour(study)

In [ ]:
optuna.visualization.plot_rank(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_terminator_improvement(study)

In [ ]:
optuna.visualization.plot_timeline(study)

In [ ]:
optuna.visualization.plot_edf(study)